### PLS algorithm for regression

In [28]:
# Importation des librairies
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score

In [35]:
# Simulation du jeux de données avec multicolinéarité 
np.random.seed(42)
n_observations = 100
X = np.zeros((n_observations, 10))
x_1 = np.random.rand(n_observations)
for i in range(10):
    x = x_1*np.random.rand(n_observations) +x_1
    X[:, i] = x

# Simulation de la variable d'intérêt 
coefficients = np.array([2, -1, 0.5, 1.5, -0.8, 1, -0.2, 0.7, -1.3, 0.9])
bruit = np.random.normal(0, 0.5, size=(n_observations,))
y = X @ coefficients + bruit

In [36]:
np.corrcoef(X, rowvar=False)
# On remarque la forte multicolinéarité entre nous variables indépendantes

array([[1.        , 0.85984859, 0.85396178, 0.8580686 , 0.88962604,
        0.86956469, 0.86570719, 0.88145586, 0.89891457, 0.89554221],
       [0.85984859, 1.        , 0.87922516, 0.88019925, 0.89632856,
        0.91367994, 0.91826875, 0.88048968, 0.89862809, 0.884083  ],
       [0.85396178, 0.87922516, 1.        , 0.89156342, 0.87038312,
        0.87107491, 0.89493286, 0.86435235, 0.87112701, 0.86578077],
       [0.8580686 , 0.88019925, 0.89156342, 1.        , 0.88678546,
        0.91609583, 0.88714371, 0.86262562, 0.87512374, 0.89698458],
       [0.88962604, 0.89632856, 0.87038312, 0.88678546, 1.        ,
        0.90711364, 0.89874567, 0.88617112, 0.90942218, 0.89438311],
       [0.86956469, 0.91367994, 0.87107491, 0.91609583, 0.90711364,
        1.        , 0.91426486, 0.8782436 , 0.88961688, 0.92600758],
       [0.86570719, 0.91826875, 0.89493286, 0.88714371, 0.89874567,
        0.91426486, 1.        , 0.9010331 , 0.90062232, 0.88760175],
       [0.88145586, 0.88048968, 0.8643523

In [37]:
# Standardisation des données
X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0, ddof=1)
X_scaled = (X - X_mean) / X_std

y_mean = np.mean(y)
y_std = np.std(y, axis=0, ddof=1)
y_scaled = y - y_mean / y_std

In [38]:
# Split en training et test set
split = 80
X_train, X_test = X_scaled[:split, :], X_scaled[split:, :]
y_train, y_test = y_scaled[:split], y_scaled[split:]

In [39]:
n_components = 5
y_train_mean = np.mean(y_train)

# Initialisation des matrices
W = np.zeros((X_train.shape[1], n_components))
Z = np.zeros((X_train.shape[0], n_components))
P = np.zeros((X_train.shape[1], n_components))
D = np.zeros((1, n_components))
B = np.zeros((X_train.shape[1], n_components))

In [40]:
#Boucle de l'algorithme

for i in range(n_components):
    # Calcul de w
    w = X_train.T @ y_train
    W[:, i] = w
    
    # Calcul de z
    z = X_train @ w
    Z[:, i] = z
    
    # Calcul de p
    p = X_train.T @ z / (z.T @ z)
    P[:, i] = p
    
    # Calcul de d
    d = y_train.T @ z / (z.T @ z)
    D[0, i] = d
    
    # Calcul de b
    b = W @ D.T
    B[:, i] = b.flatten()

    # Déflation
    X_train -= np.outer(z, p.T)
    y_train -= np.outer(z, d).flatten()

    # Prédiction sur l'ensemble de test
    y_pred = X_test @ B + y_train_mean
    
    # Évaluation du modèle
    mse = np.mean((y_test - np.mean(y_pred, axis=1)) ** 2)
    print(f"Mean Squared Error (Component {i+1}): {mse}")

Mean Squared Error (Component 1): 1.5595298389133752
Mean Squared Error (Component 2): 1.0810821825971557
Mean Squared Error (Component 3): 0.7270839818324687
Mean Squared Error (Component 4): 0.48539317637862256
Mean Squared Error (Component 5): 0.34687280754700095


### Vérification des résultats à l'aide de la fonction

In [51]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [52]:
max_comp = 5
scores = []

In [53]:
# Boucle pour identifier le nombre optimal de composantes
for n_components in range(1, max_comp + 1):
    pls = PLSRegression(n_components=n_components)
    score = np.mean(cross_val_score(pls, X_train, y_train, cv=5, scoring='neg_mean_squared_error'))
    scores.append(score)

In [54]:
optimal_components = np.argmax(scores) + 1
optimal_components

1

In [55]:
final_pls = PLSRegression(n_components=optimal_components)
final_pls.fit(X_train, y_train)

PLSRegression(n_components=1)

In [57]:
# Prédiction
y_pred = final_pls.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 2.3232196877329447


On remarque que le MSE obtenu est légèrement supérieur à celui obtenu par notre algorithme pour le même nombre de composantes à savoir 1.